In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np

In [2]:
from proc_v1 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform('data/train.csv').assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1)
)
df_test = p.transform('data/test.csv')
len(df_train)

750000

# 1. Polynomial Feature Generation & Selection

- 1차, 1 / 1차, 2차, 1차 / 1차, 2차 / 1차, 1차 / 2차, 3차 속성을 만듭니다.
- 전체 속성으로 선형 모델을 학습 시키고, 회귀계수를 봅니다.
- 다중 공선성으로 인해 회귀계수가 커진 케이스가 확인 됐습니다. VIF를 측정하여 속성을 선별합니다.

In [4]:
from itertools import combinations, product

X_r = [i + '_r' for i in X_num]
X_o2 = [(i + '_2' if i == j else i + '_mul_'+ j, (i, j)) for i, j in product(X_num, X_num) if i <= j]
X_1d1 = [(i + '_div_' + j, (i, j)) for i, j in product(X_num, X_num) if i != j]
X_o2d1 = [
    (i + '_div_' + j, (i, j)) for (i, a), j in product(X_o2, X_num) if j not in a 
]
X_o1d2 = [
    (i + '_div_' + j, (i, j)) for i, (j, a) in product(X_num, X_o2) if i not in a 
]

X_o3 = [
    (
        i + '_3' if i == j and j == k else i + '_2_mul_' + k if i == j else i + '_2_mul_' + k if i == k  else j + '_2_mul_' + i if j == k else i + '_mul_' + j + '_mul_' + k,
        (i, j, k)
    )
    for i, j, k in product(X_num, X_num, X_num) if i <= j and i <= k and j <= k
]

In [5]:
var_list = [(1 / df_train[a]).rename(a + '_r') for a in X_num]
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32')).rename(i) for i, (a, b) in X_o2])
var_list.extend([(df_train[a].astype('float32') / df_train[b].astype('float32')).rename(i) for i, (a, b) in X_1d1])
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32') * df_train[c].astype('float32')).rename(i) for i, (a, b, c) in X_o3])

df_sel = dproc.join_and_assign(
    df_train,
    pd.concat(var_list, axis = 1)
)
df_sel = dproc.join_and_assign(
    df_sel,
    pd.concat([(df_sel[a] / df_sel[b]).rename(i) for i, (a, b) in X_o2d1 + X_o1d2], axis = 1)
)
df_sel.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Calories_Log,Age_r,...,Body_Temp_div_Height_2,Body_Temp_div_Height_mul_Weight,Body_Temp_div_Weight_2,Body_Temp_div_Duration_mul_Height,Body_Temp_div_Duration_mul_Weight,Body_Temp_div_Duration_2,Body_Temp_div_Duration_mul_Heart_Rate,Body_Temp_div_Heart_Rate_mul_Height,Body_Temp_div_Heart_Rate_mul_Weight,Body_Temp_div_Heart_Rate_2
id,,,,,,,,,,,,,,,,,,,,,
0,True,36,189.0,82.0,26.0,101.0,41.000000,150.0,5.017280,0.027778,...,0.001148,0.002646,0.006098,0.008344,0.019231,0.060651,0.015613,0.002148,0.004950,0.004019
1,False,64,163.0,60.0,8.0,85.0,39.700001,34.0,3.555348,0.015625,...,0.001494,0.004059,0.011028,0.030445,0.082708,0.620313,0.058382,0.002865,0.007784,0.005495
2,False,51,161.0,64.0,7.0,84.0,39.799999,29.0,3.401197,0.019608,...,0.001535,0.003863,0.009717,0.035315,0.088839,0.812245,0.067687,0.002943,0.007403,0.005641
3,True,20,192.0,90.0,25.0,105.0,40.700001,140.0,4.948760,0.050000,...,0.001104,0.002355,0.005025,0.008479,0.018089,0.065120,0.015505,0.002019,0.004307,0.003692
4,False,38,166.0,61.0,25.0,102.0,40.599998,146.0,4.990433,0.026316,...,0.001473,0.004009,0.010911,0.009783,0.026623,0.064960,0.015922,0.002398,0.006525,0.003902


In [6]:
X_sel = [i for i in df_sel.columns if i not in ('Calories', 'Calories_Log')]

In [7]:
from sklearn.preprocessing import StandardScaler 
df_sel[X_sel] = StandardScaler().fit_transform(df_sel[X_sel])

In [ ]:
result = sgml.cv(df_sel, ss, {'X_num': X_sel}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

In [ ]:
result['model_result'][0]['coef'].sort_values()

- 변수가 300개라 모든 변수를 가지고 순차적 선택을 하는 것에는 리소스에 무리가 있었습니다.

- 이를 보완 하기 위해, 차수가 낮은 것에서 시작하여 높은 것으로 그룹핑하여 추가를 해갑니다.

> from cuml.linear_model import LinearRegression as cu_LinearRegression
> 
> lr_c_adapter = sgml.SklearnAdapter(cu_LinearRegression)
> 
> 속도를 높이기 위해 cuml.LinearRegression을 사용해봤지만, 역효과만 있습니다.

In [22]:
from tqdm.notebook import tqdm
def get_rmse(X):
    return sgml.cv(df_sel, ss, {'X_num': X}, {**config, 'progress_callback': None}, lr_adapter)['valid_scores'][0]

def fs(X, X_ff):
    return pd.Series(
        {i: get_rmse(X + [i]) for i in X_ff if i not in X}
    ).sort_values()
def be(X, floated_list):
    if len(X) < 2: return pd.Series()
    return pd.Series(
        {i: get_rmse([j for j in X if j != i]) for i in X[:-1] if i not in floated_list}
    ).sort_values()

def step_fs(X_, X_selected, floated_list, rmse_list):
    while(True):
        s_rmse = fs(X_selected, X_)
        if len(s_rmse) == 0:
            break
        if rmse_list[-1] > s_rmse.iloc[0]:
            rmse_list.append(s_rmse.iloc[0])
            X_selected.append(s_rmse.index[0])
            print("Selected: {}, RMSE: {}".format(X_selected[-1], rmse_list[-1]))
        else:
            break
        while(True):
            s_rmse = be(X_selected[:-1], floated_list)
            if len(s_rmse) > 0 and rmse_list[-1] > s_rmse.iloc[0]:
                rmse_list.append(s_rmse.iloc[0])
                floated_list.add(s_rmse.index[0])
                X_selected = [i for i in X_selected if i != s_rmse.index[0]]
                print("Excluded: {}, RMSE: {}".format(X_selected[-1], rmse_list[-1]))
            else:
                break
    return X_selected, floated_list, rmse_list

In [9]:
X_selected = list()
rmse_list = [np.inf]
X_ = X_num
X_selected, floated_list, rmse_list = sc.cache_result('fs_o1', lambda : step_fs(X_num, X_selected, set(), rmse_list), rerun = 0)
rmse_list[-1]

np.float64(0.18064835569722082)

In [10]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_r', lambda : step_fs(X_r, X_selected, set(), rmse_list), rerun = 0
)
rmse_list[-1]

np.float64(0.11890502541192026)

In [20]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_o2', lambda : step_fs([i[0] for i in X_o2], X_selected, set(), rmse_list), rerun = 0
)
rmse_list[-1]

np.float64(0.08281917544819208)

In [ ]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_1d1', lambda : step_fs([i[0] for i in X_1d1], X_selected, set(), rmse_list), rerun = 1
)
rmse_list[-1]

Selected: Duration_div_Body_Temp, RMSE: 0.0776620488597596
Excluded: Duration_div_Body_Temp, RMSE: 0.07766204753406722
Selected: Heart_Rate_div_Weight, RMSE: 0.07718973571763937


In [83]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_o2d1', lambda : step_fs([i[0] for i in X_o2d1], X_selected, set(), rmse_list), rerun = 0
)
rmse_list[-1]

np.float64(0.08358412825458636)

In [84]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_o1d2', lambda : step_fs([i[0] for i in X_o1d2], X_selected, set(), rmse_list), rerun = 0
)
rmse_list[-1]

np.float64(0.08358412825458636)

In [85]:
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_o3', lambda : step_fs([i[0] for i in X_o3], X_selected, set(), rmse_list), rerun = 0
)
rmse_list[-1]

np.float64(0.08358412825458636)

In [14]:
X_selected, rmse_list

(['Body_Temp',
  'Heart_Rate',
  'Duration',
  'Height',
  'Duration_r',
  'Heart_Rate_r',
  'Body_Temp_r',
  'Weight_r',
  'Age_r',
  'Duration_2',
  'Age_mul_Weight',
  'Heart_Rate_mul_Height',
  'Age_mul_Heart_Rate',
  'Body_Temp_mul_Duration',
  'Duration_mul_Weight',
  'Duration_mul_Height',
  'Body_Temp_mul_Weight',
  'Weight_2',
  'Body_Temp_mul_Height',
  'Heart_Rate_mul_Weight',
  'Heart_Rate_2',
  'Age_mul_Height',
  'Age_mul_Body_Temp',
  'Age_2',
  'Age_mul_Duration',
  'Height_mul_Weight'],
 [inf,
  np.float64(0.29196013462209186),
  np.float64(0.2156951337506078),
  np.float64(0.19719277658388445),
  np.float64(0.1811255747608456),
  np.float64(0.18064878864449152),
  np.float64(0.18064835569722082),
  np.float64(0.1368928785348389),
  np.float64(0.1368888058883413),
  np.float64(0.12458855799487811),
  np.float64(0.1245853566860645),
  np.float64(0.12053494384148174),
  np.float64(0.11940880647281166),
  np.float64(0.11911921746192301),
  np.float64(0.11890502541192026),

In [15]:
result = sgml.cv(df_sel, ss, {'X_num': X_selected}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

[np.float64(0.08281917544819205)]

In [75]:
result['model_result'][0]['coef'].sort_values()

Body_Temp_r      -1.667618
Body_Temp        -1.596018
Duration_2       -0.484186
Heart_Rate_r     -0.264925
Duration_r       -0.135189
Age              -0.114819
Age_r            -0.026699
Heart_Rate       -0.024870
Height_r         -0.008415
Height           -0.001109
Weight_r          0.115493
Age_mul_Weight    0.220618
Duration          1.024658
dtype: float64

In [ ]:
- 다중공선성을 VIF